In [ ]:
import sympy as sp
import QSymPy as qp

import numpy as np
import tqdm
import matplotlib.pyplot as plt


In [ ]:
system_size = 4
b_reg_size = int(np.log2(system_size))
c_reg_size = 5
num_qubits = c_reg_size + b_reg_size + 1
qc = qp.HHL(num_qubits=num_qubits, num_clbits=0)
qc.add_gate(name='H_eI', qubits_t=[num_qubits-2])
qc.add_gate(name='H', qubits_t=[num_qubits-1])
qc.assemble_symbolic_unitary()
#qc.unitary

In [ ]:
phi = qp.QuantumState(num_qubits)
#phi.state

In [ ]:
phi_prep = qc.unitary @ phi.state
#phi_prep

In [ ]:
phi_prep_0 = phi_prep.subs({phi.state[i]: 0 for i in range(1,phi.state.shape[0])})
#phi_prep_0

In [ ]:
phi_prep_0.free_symbols

In [ ]:
p = sp.Symbol('p', real=True, nonnegative=True)
pri_prep_0_h = phi_prep_0.subs({phi.state[0]: 1,
                                f'H_eI_qt{num_qubits-2}_qc_s0_p00':1/sp.sqrt(2),
                                f'H_eI_qt{num_qubits-2}_qc_s0_p10':1/sp.sqrt(2)*(1-p),
                                f'H_qt{num_qubits-1}_qc_s0_p00':1/sp.sqrt(2),
                                f'H_qt{num_qubits-1}_qc_s0_p10':1/sp.sqrt(2)})
#pri_prep_0_h

In [ ]:
hhl_op_wo_meas_wo_prep = np.load(f'QSymPy/HHL_result_Qiskit_{system_size}x{system_size}_c{c_reg_size}.npy')
hhl_op_wo_meas_wo_prep

In [ ]:
hhl_sol = hhl_op_wo_meas_wo_prep @ pri_prep_0_h
#hhl_sol = hhl_sol.subs({'I':1j})
bitstring_le = [format(i, '0' + str(num_qubits) + 'b') for i in range(2**num_qubits)]
state_dict = {key:val for key, val in zip(bitstring_le, hhl_sol)}
#print('state_dict:', state_dict)

y = np.zeros(system_size)
y = sp.tensor.array.MutableDenseNDimArray(y)
for key in tqdm.tqdm(state_dict):
    if key[-1] == '1':
        pos = int(key[0:b_reg_size],2)
        #print(state_dict[key])
        s = state_dict[key]
        sc = sp.conjugate(s)
        ss = sc*s
        #sq = sp.Abs(s)
        sq = sp.sqrt(sp.re(s)**2 + sp.im(s)**2)
        #print(sq)
        y[pos] += sq
        #print('key:', key)
        #print('pos:', pos)
y = sp.simplify(y)
print('Quantum solution, ratio elem 0/1:', y, y[0]/y[1])

In [ ]:
n = 25
p_sol_ratio = np.zeros((n,2))
p_grad_sol_ratio = np.zeros((n,2))
sol_ratio = y[0]/y[1]
grad_sol_ratio = sp.diff(sol_ratio, p)
for i, ps in enumerate(np.linspace(0.,0.1,n)):
    p_sol_ratio[i,0] = ps
    p_sol_ratio[i,1] = (sol_ratio).subs({p:ps})
    p_grad_sol_ratio[i,0] = ps
    p_grad_sol_ratio[i,1] = (grad_sol_ratio).subs({p:ps})

In [ ]:
SMALL_SIZE = 11
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

fig, ax = plt.subplots(1,2, figsize=(12, 5))

#fig.suptitle('Analysis of HHL algorithm output given probability of failure in state preparation of state $|b \\rangle $')

ax[0].plot(p_sol_ratio[:,0], p_sol_ratio[:,1], marker= 'o',
                label='HHL solution $p$')
ax[0].set_xlabel('p')
ax[0].set_ylabel('$\\frac{|x \\rangle _1}{|x \\rangle _0}$')
ax[0].scatter(p_sol_ratio[0,0], p_sol_ratio[0,1], marker='x', c='green', s=100,
                   label='correct solution')
ax[0].legend()


ax[1].plot(p_grad_sol_ratio[:,0], p_grad_sol_ratio[:,1], marker= 'o', label='gradient')
ax[1].set_xlabel('p')
ax[1].set_ylabel('$\\frac{|x \\rangle _1 / |x \\rangle _0}{\\partial p}$', fontsize=19)
ax[1].legend()

fig.tight_layout()

plt.savefig('QSymPy_HHL_p_sol_ratio.png', dpi=600, bbox_inches='tight')

In [ ]:
grad_y_0b1 = sp.diff(y[0]/y[1], p)
grad_y_0b1